# Summation Transformer Colab Runner\n\nThis notebook trains `train_strictish_h2_d6_singlecarry_from_scratch.py` and saves artifacts to `/content/outputs` (Colab-safe path).

In [ ]:
import os\n\nREPO_URL = "https://github.com/Deferf/Summation-Transformer.git"\nREPO_DIR = "/content/Summation-Transformer"\nOUT_DIR = "/content/outputs"\n\nif not os.path.exists(REPO_DIR):\n    !git clone {REPO_URL}\n\n%cd /content/Summation-Transformer\n!pip -q install -r requirements.txt\n!mkdir -p {OUT_DIR}

In [ ]:
import torch\n\ndevice = "cuda" if torch.cuda.is_available() else "cpu"\nprint("Using device:", device)\n\nplot_path = "/content/outputs/strictish_h2_d6_singlecarry_colab_metrics.png"\nckpt_path = "/content/outputs/strictish_h2_d6_singlecarry_colab.pt"\n\n!python train_strictish_h2_d6_singlecarry_from_scratch.py \\n  --device {device} \\n  --steps 1200 \\n  --batch-size 256 \\n  --log-every 100 \\n  --eval-samples 256 \\n  --final-eval-samples 2000 \\n  --target-acc 0.999 \\n  --checkpoint {ckpt_path} \\n  --plot-path {plot_path}

In [ ]:
from IPython.display import Image, display\n\nplot_path = "/content/outputs/strictish_h2_d6_singlecarry_colab_metrics.png"\ndisplay(Image(filename=plot_path))

Optional: download artifacts from `/content/outputs/` using Colab file browser or with `files.download(...)`.